<a href="https://www.kaggle.com/code/tanvikiran27/bases-for-spherical-harmonics?scriptVersionId=248712598" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import itertools as it
import sympy as sp 
import pandas as pd
from sympy import symbols, Function, diff, FiniteSet, simplify, re, AlgebraicNumber
from sympy import Q, ask
from sympy import *
from sympy.utilities.iterables import partitions
from sympy.functions.combinatorial.numbers import nC
from sympy.matrices import diag, eye

import os
import zipfile
import shutil

# Bases for spherical harmonics

$n$ = dimension ($\mathbb{R}^n$ or $\mathbb{C}^n$) \
$k$ = real degree \
$(p,q)$ = complex bidegree

At the beginning of a session, input your values for $n$, $k$, $p$, and $q$ below and click Run All to generate a basis. Then for every new basis, just make sure to run the cell below each time you change the values.

In [2]:
####### Change constants here #######
n = 3
k = 3
p = 1
q = 2
#####################################

# Setting up symbols
f = symbols('f', cls = Function)
x,y,z,w,zeta = symbols('x y z w zeta')
zs = symbols(f'z1:{n+1}', complex=True)
zbar = symbols(f'zbar1:{n+1}', complex=True)
ls = symbols(f'l1:{n+1}', int=True)

# Newton potential (R^n) 
mag_real = 0
for i in range(n):
    mag_real += zs[i]**2 ## Euclidean norm squared
np_real = sp.sqrt(mag_real)**(2-n)

# Newton potential (C^n)
mag_complex = 0
for i in range(n):
    mag_complex += zs[i]*zbar[i] ## Complex modulus squared
np_complex = sp.sqrt(mag_complex)**(2-2*n)

To generate a basis, make a new cell and call the function corresponding to the type of basis you want. 

real_basis() will give you a basis for $H_k$ on the sphere in $\mathbb{R}^n$ \
basis_H_pq() will give you a basis for $H_{p,q}$ on the sphere in $\mathbb{C}^n$ (this is probably the one you'll use most)

In [3]:
# Multi index differentiation
def D(f:sp.Add, alpha: sp.FiniteSet, zs:[sp.symbols]) -> sp.Add:
    df = f
    for i in range(n):
        df = diff(df,(zs[i],alpha[i]))
    return df

In [4]:
# Returns valid n-tuples
# https://www.geeksforgeeks.org/python-program-to-convert-dictionary-to-list-by-repeating-keys-corresponding-value-times/
def get_alpha_beta(deg:int) -> sp.FiniteSet:
    alphas = FiniteSet()
    for part in partitions(deg, m=n):
        part = list(it.chain.from_iterable(it.repeat(k, v) for k, v in part.items()))
        while len(part) < n:
            part.append(0)
        for perm in it.permutations(part):
            alphas += FiniteSet(perm)
    return alphas

In [5]:
# Turns rational functions in our basis into polynomials
def make_polynomial(f:sp.Function, mag:sp.Add) -> sp.Add:
    f = sp.numer(f).subs(mag, x)
    while f.is_polynomial(zs,zbar,x) == False:
        f = simplify(f * x)
        f = sp.numer(f).subs(mag, x)
    return expand(f.subs(x,abs(z)**2))

In [6]:
# For H_k(R^n) *NOT C^n
def real_dim() -> int:
    if k-2<0:
        return nC(n+k-1,k)
    else:
        return nC(n+k-1,k) - nC(n+k-3,k-2)

In [7]:
# For H_k(C^n)
def dim_H_k() -> int:
    if k-2<0:
        return nC(2*n+k-1,k)
    else:
        return nC(2*n+k-1,k) - nC(2*n+k-3,k-2)

In [8]:
# For H_p,q(C^n)
def dim_H_pq() -> int:
    if p-1<0 or q-1<0:
        return nC(n+p-1,p)*nC(n+q-1,q)
    else:
        return nC(n+p-1,p)*nC(n+q-1,q) - nC(n+p-2,p-1)*nC(n+q-2,q-1)

In [9]:
# Finding basis for H_k(R^n) - see Axler's Harmonic Function Theory p. 92
def real_basis() -> sp.FiniteSet:
    basis = FiniteSet()
    alphas = get_alpha_beta(k)
    for a in alphas:
        if a[0] > 1:
            alphas -= FiniteSet(a)
    print(f'# of elements = {len(alphas)}')
    print(f'Dimension from combinatorial expression = {real_dim()} \n')
    for a in alphas:
        d = D(np_real, a, zs)
        d = make_polynomial(d, mag_real)
        print(d, '\n')
        basis += FiniteSet(d)
    return basis

In [10]:
# Finding basis for H_p,q(C^n)
def basis_H_pq() -> sp.FiniteSet:
    basis = FiniteSet()
    alphas = get_alpha_beta(p)
    betas = get_alpha_beta(q)
    ab = FiniteSet()
    for a in alphas:
        for b in betas:
            if a[0]==0 or b[0]==0:
                ab += FiniteSet((a,b))
    print(f'# of elements = {len(ab)}')
    print(f'Dimension from combinatorial expression = {dim_H_pq()} \n')
    for (a,b) in ab:
        d = D(np_complex,b,zs)
        d = D(d,a,zbar)
        d = make_polynomial(d, mag_complex)
        print(d, '\n')
        basis += FiniteSet(d)
    return basis

In [11]:
# Regular complex Laplacian
def box(f:sp.Add) -> sp.Add:
    laplacian = 0
    for i in range(n):
        laplacian += diff(diff(f,zbar[i]),zs[i])
    return laplacian

In [12]:
# Kohn Laplacian as given in 2018 REU paper
def box_b(f:sp.Add) -> sp.Add:
    box_b = 0
    # first term
    for i in range(n): 
        box_b += -2*diff(diff(f,zbar[i]), zs[i])
        
    # second term
    for i in range(n): 
        box_b += 2*diff(zs[i]*q*f, zs[i])
        
    # third term (double sum)
    for i in range(n): 
        for a in range(n):
            box_b += 2*diff(zs[a]*zbar[i]*diff(f,zbar[i]), zs[a])
            
    # fourth term (double sum)
    for i in range(n): 
        for a in range(n):
            box_b += -2*diff(zs[a]*zbar[i]*zs[i]*q*f, zs[a])
    return box_b

In [13]:
# Just in case :)
# Checks whether a given function is harmonic
def is_harmonic(f:sp.Add) -> bool:
    return (box_b(f)/f).simplify().is_polynomial(mag_complex)

In [14]:
# Adapted from SoS 2020 unfinished REU paper
def M(j:int, k:int, f:sp.Add) -> sp.Add:
    return zbar[j]*diff(f,zs[k]) - zbar[k]*diff(f,zs[j])

def M_bar(j:int, k:int, f:sp.Add) -> sp.Add:
    return zs[j]*diff(f,zbar[k]) - zs[k]*diff(f,zbar[j])

# Equivalent to Kohn Laplacian
def R(f:sp.Add) -> sp.Add:
    R = 0
    for k in range(1,n):
        for j in range(k):
            R += M(j,k,M_bar(j,k,f))
    R *= -1
    return R